# Data Processing & Cleaning

This notebook implements the "minimal safe cleaning" and feature engineering steps for the Movie Hit vs Flop classifier.

## Steps:
1. Load Data
2. Basic Cleaning (Duplicates, Zeros)
3. Label Creation (ROI -> Hit/Flop)
4. Feature Engineering (Dates, JSON columns)
5. Leakage Removal
6. Save Processed Data

In [2]:
import pandas as pd
import numpy as np
import json
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')

## 1. Load Data

In [3]:
# Load datasets
credits = pd.read_csv('../data/raw/tmdb_5000_credits.csv')
movies = pd.read_csv('../data/raw/tmdb_5000_movies.csv')

# Merge datasets on 'id'
# Credits has 'movie_id' which corresponds to 'id' in movies
credits.rename(columns={'movie_id': 'id'}, inplace=True)
df = movies.merge(credits, on='id')

print(f"Initial shape: {df.shape}")
df.head(2)

Initial shape: (4803, 23)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


## 2. Minimal Safe Cleaning

In [4]:
# Drop duplicates on ID
df = df.drop_duplicates(subset=['id'])

# Drop rows where budget or revenue is <= 0 (unknowns)
df = df[(df['budget'] > 0) & (df['revenue'] > 0)]

# Filter for runtime (strictly feature films: 40 to 240 mins)
df = df[(df['runtime'] >= 40) & (df['runtime'] <= 240)]

print(f"Shape after cleaning: {df.shape}")

Shape after cleaning: (3227, 23)


## 3. Define Label (Hit vs Flop)

In [5]:
# Calculate ROI
df['roi'] = df['revenue'] / df['budget']

# Define Label
# Hit: ROI >= 2.0
# Flop: ROI < 1.0
# Drop middle band (1.0 <= ROI < 2.0)

def get_label(roi):
    if roi >= 2.0:
        return 'Hit'
    elif roi < 1.0:
        return 'Flop'
    else:
        return 'Exclude'

df['label'] = df['roi'].apply(get_label)

# Filter out the 'Exclude' band
df = df[df['label'] != 'Exclude']

print(f"Shape after label creation: {df.shape}")
print(df['label'].value_counts())

Shape after label creation: (2596, 25)
label
Hit     1808
Flop     788
Name: count, dtype: int64


## 4. Feature Engineering

In [6]:
# A) Dates
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_month'] = df['release_date'].dt.month

# B) Parse JSON columns
# Genres
df['genres'] = df['genres'].apply(literal_eval)
df['genre_names'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# Production Companies
df['production_companies'] = df['production_companies'].apply(literal_eval)
df['company_names'] = df['production_companies'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# Multi-hot encode Genres
genres_dummies = pd.get_dummies(df['genre_names'].apply(pd.Series).stack()).groupby(level=0).sum()
df = pd.concat([df, genres_dummies], axis=1)

# Top Production Companies (Top 20)
all_companies = [c for sublist in df['company_names'] for c in sublist]
top_companies = pd.Series(all_companies).value_counts().head(20).index.tolist()

for company in top_companies:
    df[f'company_{company}'] = df['company_names'].apply(lambda x: 1 if company in x else 0)

# C) Log Transform Budget
df['log_budget'] = np.log1p(df['budget'])

## 5. Remove Leakage & Final Selection

In [7]:
# Columns to drop (post-release or leakage)
leakage_cols = [
    'revenue', 'vote_average', 'vote_count', 'popularity', 'roi', 
    'status', 'original_title', 'title_x', 'title_y',  # Redundant or text only
    'genres', 'genre_names', 'production_companies', 'company_names', # Processed versions kept
    'overview', 'tagline', 'homepage', 'keywords', 'spoken_languages', 'production_countries' # Text/Complex features for now
]

# Keep basic metadata + features
keep_cols = ['id', 'budget', 'log_budget', 'runtime', 'release_date', 'release_month', 'label'] + list(genres_dummies.columns) + [f'company_{c}' for c in top_companies]

# Drop leakage cols that are in df
cols_to_drop = [c for c in leakage_cols if c in df.columns]
df_clean = df.drop(columns=cols_to_drop)

# Alternatively, just select what we want
# df_model = df[keep_cols] # Use this if we want to be strict

print(f"Final shape: {df_clean.shape}")
df_clean.head()

Final shape: (2596, 49)


,budget,id,original_language,release_date,runtime,cast,crew,label,release_month,Action,...,company_United Artists,company_DreamWorks SKG,company_Miramax Films,company_Village Roadshow Pictures,company_Dune Entertainment,company_Regency Enterprises,company_Amblin Entertainment,company_TriStar Pictures,company_Working Title Films,log_budget
0,237000000,19995,en,2009-12-10,162.0,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Hit,12,1.0,...,0,0,0,0,1,0,0,0,0,19.283571
1,300000000,285,en,2007-05-19,169.0,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Hit,5,1.0,...,0,0,0,0,0,0,0,0,0,19.519293
2,245000000,206647,en,2015-10-26,148.0,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Hit,10,1.0,...,0,0,0,0,0,0,0,0,0,19.316769
3,250000000,49026,en,2012-07-16,165.0,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Hit,7,1.0,...,0,0,0,0,0,0,0,0,0,19.336971
5,258000000,559,en,2007-05-01,139.0,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de...",Hit,5,1.0,...,0,0,0,0,0,0,0,0,0,19.368470


## 6. Save Processed Data

In [8]:
output_path = '../data/processed/train_processed.csv'
df_clean.to_csv(output_path, index=False)
print(f"Saved processed data to {output_path}")

Saved processed data to ../data/processed/train_processed.csv
